## Feature Engineering and CV based on 5-th Place Solutions

`param = {'objective': 'multi:softprob', 
         'eta': 0.05, 
         'max_depth': 8,
         'silent': 1, 
         'num_class': len(target_cols),
         'eval_metric': 'mlogloss',
         'min_child_weight': 1,
         'subsample': 0.7,
         'colsample_bytree': 0.7,
         'seed': 0}`
         
`n_repeats=2
n_trees = 150`

val-MAP@7:0.89935, private LB: 0.0266884, public LB: 0.0264044

New in this notebook:
- Create training data for all months (2015-02-28 to 2016-04-28) and validate on 2016-05-28

In [1]:
from santander_helper import *

Create all months' data

In [2]:
# fixed_lag = 6
# for i, m in tqdm.tqdm_notebook(enumerate(month_list), total=len(month_list)):
#     print(m)
#     if m in ['2015-01-28', '2016-06-28']:
#         continue
#     x_train, y_train, w_train = create_train(m, max_lag=i, fixed_lag=fixed_lag, pattern_flag=True)
#     print('-'*60)
# del x_train, y_train, w_train
# gc.collect()

Load all months' data

In [3]:
x_train = []
y_train = []
w_train = []
fixed_lag = 6
for i, m in tqdm.tqdm_notebook(enumerate(month_list), total=len(month_list)):
    if m in ['2015-01-28', '2016-06-28']:
        continue
    x_tmp, y_tmp, w_tmp = create_train(m, max_lag=i, fixed_lag=fixed_lag, pattern_flag=True)
    x_train.append(x_tmp)
    y_train.append(y_tmp)
    w_train.append(w_tmp)
del x_tmp, y_tmp, w_tmp
gc.collect()

# Prepare for train and validation
x_val = x_train[-1]
y_val = y_train[-1]
w_val = w_train[-1]

x_train = pd.concat(x_train[:-1], axis=0, ignore_index=True, sort=False)
y_train = pd.concat(y_train[:-1], axis=0, ignore_index=True, sort=False)
w_train = pd.concat(w_train[:-1], axis=0, ignore_index=True, sort=False)

gc.collect()

203

In [4]:
param = {'objective': 'multi:softprob', 
         'eta': 0.1, 
         'max_depth': 8,
         'silent': 1, 
         'num_class': len(target_cols),
         'eval_metric': 'merror',
         'min_child_weight': 10,
         'lambda': 5,
         'subsample': 0.7,
         'colsample_bytree': 0.7,
         'seed': 0}

n_rows = None # number of rows in train dataset, to simplify testing, always set to None
n_repeats = 1
n_trees = 50
train = {'x': x_train.iloc[:n_rows, :], 'y': y_train.iloc[:n_rows], 'w': w_train.iloc[:n_rows]}
val = {'x': x_val.iloc[:n_rows, :], 'y': y_val.iloc[:n_rows], 'w': w_val.iloc[:n_rows]}
df, clfs, running_time = cv_all_month(param, train, val, n_features=350, num_boost_round=n_trees, 
    n_repeats=n_repeats, random_state=0, verbose_eval=True)

[0]	train-mlogloss:2.66292	val-mlogloss:2.67474	train-MAP@7:0	val-MAP@7:0.893116
[1]	train-mlogloss:2.46916	val-mlogloss:2.486	train-MAP@7:0	val-MAP@7:0.897498
[2]	train-mlogloss:2.31823	val-mlogloss:2.33769	train-MAP@7:0	val-MAP@7:0.898923
[3]	train-mlogloss:2.19381	val-mlogloss:2.21584	train-MAP@7:0	val-MAP@7:0.899542
[4]	train-mlogloss:2.08748	val-mlogloss:2.11122	train-MAP@7:0	val-MAP@7:0.900673
[5]	train-mlogloss:1.99476	val-mlogloss:2.01956	train-MAP@7:0	val-MAP@7:0.901178
[6]	train-mlogloss:1.91287	val-mlogloss:1.93849	train-MAP@7:0	val-MAP@7:0.901897
[7]	train-mlogloss:1.83969	val-mlogloss:1.8672	train-MAP@7:0	val-MAP@7:0.902293
[8]	train-mlogloss:1.77395	val-mlogloss:1.80178	train-MAP@7:0	val-MAP@7:0.902894
[9]	train-mlogloss:1.7143	val-mlogloss:1.7428	train-MAP@7:0	val-MAP@7:0.903168
[10]	train-mlogloss:1.65943	val-mlogloss:1.68941	train-MAP@7:0	val-MAP@7:0.903383
[11]	train-mlogloss:1.60891	val-mlogloss:1.63917	train-MAP@7:0	val-MAP@7:0.903694
[12]	train-mlogloss:1.56214	val

[99]	train-mlogloss:0.74372	val-mlogloss:0.771048	train-MAP@7:0	val-MAP@7:0.910264
[100]	train-mlogloss:0.742555	val-mlogloss:0.769932	train-MAP@7:0	val-MAP@7:0.910319
[101]	train-mlogloss:0.741441	val-mlogloss:0.76881	train-MAP@7:0	val-MAP@7:0.910422
[102]	train-mlogloss:0.740364	val-mlogloss:0.767745	train-MAP@7:0	val-MAP@7:0.910517
[103]	train-mlogloss:0.739239	val-mlogloss:0.766603	train-MAP@7:0	val-MAP@7:0.910583
[104]	train-mlogloss:0.738224	val-mlogloss:0.76562	train-MAP@7:0	val-MAP@7:0.910527
[105]	train-mlogloss:0.73721	val-mlogloss:0.764652	train-MAP@7:0	val-MAP@7:0.910671
[106]	train-mlogloss:0.736178	val-mlogloss:0.76371	train-MAP@7:0	val-MAP@7:0.910706
[107]	train-mlogloss:0.735202	val-mlogloss:0.762874	train-MAP@7:0	val-MAP@7:0.910689
[108]	train-mlogloss:0.734225	val-mlogloss:0.761904	train-MAP@7:0	val-MAP@7:0.910715
[109]	train-mlogloss:0.733256	val-mlogloss:0.76098	train-MAP@7:0	val-MAP@7:0.910757
[110]	train-mlogloss:0.732339	val-mlogloss:0.760014	train-MAP@7:0	val-MA

KeyboardInterrupt: 

In [ ]:
x_test = create_test(month='2016-06-28', max_lag=17, fixed_lag=6, pattern_flag=True)

In [ ]:
simulation_name = 'p1'
y_pred, y_sub = predict_all_month(model_dict=clfs, x_test=x_test, 
    sub_name='eda_4_33_{}.csv.gz'.format(simulation_name), n_features=350)